In [1]:
# 主要步驟
#     讀取資料
#         讀檔案
#         取出常用變數(若有更改常用變數時要記得更新
#     清洗資料
#         刪除價格值為0的奇怪資料
#         刪除不會用到的欄位
#         刪除不會用到的列
#         找出資料誤值或空值
#         驗算資料
#     轉換資料
#         修正日期格式
#         以代號取代原資料(市場>>代號、花名重編、分類)
#         將各種花合併得到總交易量、平均價格
#         新增已周圍單位的存檔
#     存取資料

In [2]:
# seting 讀取檔案
import os, glob
import numpy as np
import pandas as pd
from datetime import date, time, datetime
from datetime import timedelta

fn_list = glob.glob("../raw_f_price/*.csv")
# print(fn_list)
# fn = fn_list[4]


In [3]:
def merge_flower_price(fn):
    df = pd.read_csv(fn, encoding="utf-8")
    flower = fn.split("\\")[-1].replace("all","").replace(".csv", "")
    print(flower)
    # 刪除不使用的資料 
    df = df.drop(["增減%","增減%.1","殘貨量"], axis=1)
    # 總行數 update
    nraw = df.shape[0]
    # 個別欄位名稱 update
    c = df.columns.tolist()
    # 資料檢查 part 1
    for colum in ['市場', '產品']:
        freq = df[colum].value_counts()
        dist_value = df[colum].drop_duplicates()
        null_data = df.iloc[df[colum].index.isnull()]
    flo_price = np.array(df["平均價"])
    # 將要刪除的index 放進去
    ind_to_delete=[]
    n = len(flo_price)
    for i in range(n):
        if int(flo_price[i])==0:
            ind_to_delete.append(i)
    # print("delete:", len(ind_to_delete))
    df = df.drop(ind_to_delete,axis=0)
    df.reset_index(drop=True)
    nraw = df.shape[0]
    date_old = np.array(df["日期"])
    date_new = []
    dates = len(date_old)
    # dates = 10
    for i in range(dates):
        y,m,d = date_old[i].split("/")
        n_date = str(int(y)+1911) + "-" + m + "-" + d
        date_new.append(n_date)
    date_time_new = np.asarray(date_new)
    df["日期"] = date_time_new
    # 新稱欄位為year 做 group by
    d = np.array(df["日期"])
    year = []
    dates = len(d)
    for i in range(dates):
        date_data = d[i].split("-")
        year.append(date_data[0])
    df["year"] = year
    c = df.columns.tolist()
    # 每年groyp by 取前30名
    gdf  = df.groupby("year")
    year = ["2014", "2015", "2016", "2017", "2018"]
    df_by_year=[]
    year_rank_30=[]
    for y in year:
        t_df = gdf.get_group(y)
        freq = t_df["產品"].value_counts()
        dist_value = df[colum].drop_duplicates()
        # print(freq)
        # print(dist_value)
        r30 = set()
        for i in range(30):
            count = freq[i]
            item = freq.index[i]
    #         print(i,count, item)
            r30.add(item)
        year_rank_30.append(r30)
    # 取各年交集
    for i in range(len(year_rank_30)):
        if i==0:
            all_rank30 = year_rank_30[i]
        all_rank30 = all_rank30.intersection(year_rank_30[i])
    # print(len(all_rank30),list(all_rank30))
    finall_class = list(all_rank30)
    # 取出finall_class 的資料
    df_arr = np.array(df)
    n = len(df_arr)
    index_in_final=[]
    for i in range(n):
        if df_arr[i,2] in finall_class:
    #         print(i, df_arr[i,2])
            index_in_final.append(i)
    df_arr = df_arr[index_in_final,:]
    df= pd.DataFrame(df_arr, columns=c)
    # 移除 多於資訊
    df = df.drop(columns=["產品", "最高價", "上價", "中價", "下價"])
    # 簡短市場資訊
    mk = np.array(df["市場"])
    new_m = []
    for i in range(len(mk)):
        new_m.append(mk[i][4:6])
    df["市場"] = new_m
    # 計算個別市場每日價格
    data_r = np.array(df)
    new_data = []
    new_colums = ["日期", "市場", "總平均", "總量", "年份", "週別"]
    all_date = df["日期"].drop_duplicates()
    all_market = df["市場"].drop_duplicates()
    # data[:,:5]

    def data_trans(market, d_date):
        sub_data = data_r[(data_r[:,0]==d_date)*(data_r[:,1]==market),:]
        year = int(d_date.split("-")[0])
        d_day = date(year,int(d_date.split("-")[1]), int(d_date.split("-")[2]))- date(year, 1, 1)
        d_w = 1+ (d_day.days // 7)
        d_sum = sum(sub_data[:,3])
        if d_sum ==0:
            return [d_date, market, 0, 0, year, d_w]
        # 計算平均價格 總量
        try:
            d_avg = round(sum(sub_data[:,3]*sub_data[:,2])/d_sum,2)
        except ZeroDivisionError:
            print(d_sum, sub_data)

        # 轉換日期 得知週別
        return [d_date, market, d_avg, d_sum, year, d_w]

    for market in all_market:
        day_data =[]
        for d_date in all_date:
            d = data_trans(market, d_date)
            if d[3]>0:
                day_data.append(d)
        df_tmp = pd.DataFrame(day_data, columns=new_colums)
        fn ="../result/merge_flower_price"flower + "_" + market + ".csv"
        df_tmp.to_csv(fn, index=False, encoding="utf-8", header=False)
        print(fn, "saved")

In [4]:
for fn in fn_list:
    merge_flower_price(fn)

Anthurium
Anthurium_台北.csv saved
Anthurium_台中.csv saved
Anthurium_彰化.csv saved
Anthurium_高雄.csv saved
Anthurium_台南.csv saved
Chrysanthemum
Chrysanthemum_台北.csv saved
Chrysanthemum_台南.csv saved
Chrysanthemum_高雄.csv saved
Chrysanthemum_彰化.csv saved
Chrysanthemum_台中.csv saved
Eustoma
Eustoma_台北.csv saved
Eustoma_台中.csv saved
Eustoma_高雄.csv saved
Eustoma_彰化.csv saved
Eustoma_台南.csv saved
OrientalLily
OrientalLily_台北.csv saved
OrientalLily_彰化.csv saved
OrientalLily_台南.csv saved
OrientalLily_高雄.csv saved
OrientalLily_台中.csv saved
Rose
Rose_台北.csv saved
Rose_台中.csv saved
Rose_彰化.csv saved
Rose_台南.csv saved
Rose_高雄.csv saved
